# 📊 VisualizationTool - Discover OpenSearch Dashboards Visualizations

```mermaid
%%{init: {'theme':'base', 'themeVariables': { 'primaryColor':'#8E44AD', 'primaryTextColor':'#fff', 'primaryBorderColor':'#7D3C98', 'lineColor':'#F39C12', 'secondaryColor':'#3498DB', 'tertiaryColor':'#27AE60', 'fontSize':'16px'}}}%%
graph TB
    A[👤 User Query<br/>Show revenue dashboard] --> B[🤖 Flow Agent]
    B --> C{📊 VisualizationTool}
    C --> D[🔍 Search .kibana Index]
    D --> E[📈 Find Visualizations]
    E --> F[🎨 Match by Keywords]
    F --> G[📤 Visualization List]
    
    style A fill:#3498DB,stroke:#2980B9,color:#fff
    style C fill:#8E44AD,stroke:#7D3C98,color:#fff
    style D fill:#E67E22,stroke:#D35400,color:#fff
    style E fill:#9B59B6,stroke:#8E44AD,color:#fff
    style G fill:#27AE60,stroke:#229954,color:#fff
```

## 📚 Learning Objectives

In this notebook, you'll learn:
1. ✅ How to use **VisualizationTool** to find saved dashboards and visualizations
2. ✅ Understanding OpenSearch Dashboards' **.kibana** index structure
3. ✅ Searching for visualizations by keywords and relevance
4. ✅ Integrating visualization discovery into agent workflows
5. ✅ Best practices for dashboard management

---

## 🎯 What is VisualizationTool?

**VisualizationTool** searches the OpenSearch Dashboards **.kibana** index to find saved visualizations and dashboards that match a user's query. This is useful for:
- 📈 **Dashboard Discovery**: Find relevant visualizations for business questions
- 🔍 **Self-Service Analytics**: Help users locate existing reports
- 🎨 **Visualization Recommendation**: Suggest appropriate charts for queries
- 📊 **BI Integration**: Connect agents to dashboard content

**Key Features**:
- Searches OpenSearch Dashboards saved objects
- Matches visualizations by title, description, and metadata
- Returns ranked results by relevance
- Configurable result size
- No LLM required (simple tool)

---

## Step 1: Import Required Libraries

In [1]:
import sys
import json
from datetime import datetime

# Add parent directory to path to import helper functions
sys.path.append('..')
from agent_helpers import (
    get_os_client,
    create_flow_agent,
    execute_agent,
    cleanup_resources
)

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


## Step 2: Initialize OpenSearch Client

In [3]:
# Initialize OpenSearch client
client = get_os_client()

# Verify connection
info = client.info()
print(f"✅ Connected to OpenSearch cluster: {info['cluster_name']}")
print(f"📊 Version: {info['version']['number']}")

✅ Connected to OpenSearch cluster: docker-cluster
📊 Version: 3.3.0


## Step 3: Create Mock Visualization Metadata

In a real scenario, OpenSearch Dashboards creates these automatically. For demonstration, we'll create sample visualization metadata.

In [4]:
# Note: In production, .kibana index is managed by OpenSearch Dashboards
# We're creating sample data for demonstration purposes

kibana_index = ".kibana_demo"

# Delete if exists
if client.indices.exists(index=kibana_index):
    client.indices.delete(index=kibana_index)

# Create mock .kibana index
client.indices.create(index=kibana_index)
print(f"✅ Created demo index: {kibana_index}")

# Sample visualization objects (mimicking OpenSearch Dashboards structure)
visualizations = [
    {
        "type": "visualization",
        "visualization": {
            "title": "Monthly Revenue Dashboard",
            "description": "Total revenue by month with trend analysis",
            "visState": json.dumps({"type": "line"})
        }
    },
    {
        "type": "visualization",
        "visualization": {
            "title": "Sales by Region",
            "description": "Geographic distribution of sales across regions",
            "visState": json.dumps({"type": "pie"})
        }
    },
    {
        "type": "visualization",
        "visualization": {
            "title": "Customer Acquisition Funnel",
            "description": "Conversion rates from lead to customer",
            "visState": json.dumps({"type": "funnel"})
        }
    },
    {
        "type": "visualization",
        "visualization": {
            "title": "Product Performance Metrics",
            "description": "Top selling products with revenue and quantity metrics",
            "visState": json.dumps({"type": "bar"})
        }
    },
    {
        "type": "visualization",
        "visualization": {
            "title": "Real-time Order Monitoring",
            "description": "Live dashboard showing current orders and fulfillment status",
            "visState": json.dumps({"type": "metric"})
        }
    },
    {
        "type": "visualization",
        "visualization": {
            "title": "Customer Satisfaction Trends",
            "description": "NPS scores and customer feedback over time",
            "visState": json.dumps({"type": "area"})
        }
    },
    {
        "type": "dashboard",
        "dashboard": {
            "title": "Executive Summary Dashboard",
            "description": "High-level KPIs for executive team including revenue, growth, and customer metrics"
        }
    },
    {
        "type": "visualization",
        "visualization": {
            "title": "Inventory Levels by Warehouse",
            "description": "Current stock levels across all warehouse locations",
            "visState": json.dumps({"type": "heatmap"})
        }
    }
]

# Index visualizations
for viz in visualizations:
    client.index(index=kibana_index, body=viz, refresh=True)

print(f"✅ Created {len(visualizations)} sample visualizations")

✅ Created demo index: .kibana_demo
✅ Created 8 sample visualizations


## Step 4: Create Flow Agent with VisualizationTool

In [6]:
# Define the tool configuration
tools = [
    {
        "type": "VisualizationTool",
        "parameters": {
            "index": kibana_index,  # Point to our demo index
            "input": "${parameters.question}",
            "size": 3  # Return top 3 most relevant visualizations
        }
    }
]

# Create the flow agent
agent_id = create_flow_agent(
    client=client,
    agent_name="Visualization_Discovery_Agent",
    description="An agent that helps users find relevant dashboards and visualizations",
    tools=tools
)

print(f"✅ Flow agent created with ID: {agent_id}")
print(f"🔧 Tool configured: VisualizationTool")
print(f"📊 Searching index: {kibana_index}")

   Registering flow agent: Visualization_Discovery_Agent...
   ✓ Agent registered: irVQb5oBFJiTVjgyIJWg
✅ Flow agent created with ID: irVQb5oBFJiTVjgyIJWg
🔧 Tool configured: VisualizationTool
📊 Searching index: .kibana_demo


## Step 5: Test Case 1 - Find Revenue Dashboards

In [7]:
# Search for revenue-related visualizations
parameters = {
    "question": "what's the revenue for today?"
}

print("❓ Question: What's the revenue for today?")
print("="*60)

response = execute_agent(client, agent_id, parameters)

print("\n📊 Found Visualizations:")
print(json.dumps(response, indent=2))

❓ Question: What's the revenue for today?

📊 Found Visualizations:
{
  "inference_results": [
    {
      "output": [
        {
          "name": "response",
          "result": "Title,Id\nMonthly Revenue Dashboard,gLVPb5oBFJiTVjgyHZUq\n"
        }
      ]
    }
  ]
}


## Step 6: Test Case 2 - Find Customer Analytics

In [8]:
# Search for customer-related dashboards
parameters = {
    "question": "show me customer satisfaction metrics"
}

print("❓ Question: Show me customer satisfaction metrics")
print("="*60)

response = execute_agent(client, agent_id, parameters)

print("\n📈 Customer Visualizations:")
print(json.dumps(response, indent=2))

❓ Question: Show me customer satisfaction metrics

📈 Customer Visualizations:
{
  "inference_results": [
    {
      "output": [
        {
          "name": "response",
          "result": "Title,Id\nCustomer Satisfaction Trends,hrVPb5oBFJiTVjgyHZVj\nProduct Performance Metrics,hLVPb5oBFJiTVjgyHZVU\nCustomer Acquisition Funnel,g7VPb5oBFJiTVjgyHZVN\n"
        }
      ]
    }
  ]
}


## Step 7: Test Case 3 - Find Sales Analysis

In [9]:
# Search for sales and product performance
parameters = {
    "question": "which products are selling best?"
}

print("❓ Question: Which products are selling best?")
print("="*60)

response = execute_agent(client, agent_id, parameters)

print("\n🛍️ Product Sales Visualizations:")
print(json.dumps(response, indent=2))

❓ Question: Which products are selling best?

🛍️ Product Sales Visualizations:
{
  "inference_results": [
    {
      "output": [
        {
          "name": "response",
          "result": "No Visualization found"
        }
      ]
    }
  ]
}


## Step 8: Test Case 4 - Find Executive Dashboards

In [10]:
# Search for high-level executive dashboards
parameters = {
    "question": "show me the executive summary dashboard"
}

print("❓ Question: Show me the executive summary dashboard")
print("="*60)

response = execute_agent(client, agent_id, parameters)

print("\n👔 Executive Dashboards:")
print(json.dumps(response, indent=2))

❓ Question: Show me the executive summary dashboard

👔 Executive Dashboards:
{
  "inference_results": [
    {
      "output": [
        {
          "name": "response",
          "result": "Title,Id\nMonthly Revenue Dashboard,gLVPb5oBFJiTVjgyHZUq\n"
        }
      ]
    }
  ]
}


## Step 9: Test Case 5 - Find Operational Dashboards

In [11]:
# Search for real-time operational monitoring
parameters = {
    "question": "show me real-time order status"
}

print("❓ Question: Show me real-time order status")
print("="*60)

response = execute_agent(client, agent_id, parameters)

print("\n⏱️ Operational Monitoring Visualizations:")
print(json.dumps(response, indent=2))

❓ Question: Show me real-time order status

⏱️ Operational Monitoring Visualizations:
{
  "inference_results": [
    {
      "output": [
        {
          "name": "response",
          "result": "Title,Id\nReal-time Order Monitoring,hbVPb5oBFJiTVjgyHZVc\n"
        }
      ]
    }
  ]
}


## 🎓 Key Takeaways

### What We Learned:

1. **VisualizationTool Capabilities**:
   - ✅ Searches OpenSearch Dashboards saved objects
   - ✅ Matches visualizations by title and description
   - ✅ Returns ranked results by relevance
   - ✅ Configurable result size
   - ✅ No LLM required (keyword-based search)

2. **How It Works**:
   - Queries the **.kibana** index (or custom index)
   - Uses text matching on visualization metadata
   - Ranks results by relevance score
   - Returns visualization titles, descriptions, and IDs

3. **Practical Use Cases**:
   - 📊 **Self-Service BI**: Users find dashboards without knowing exact names
   - 🤖 **Agent-Powered Analytics**: Agents recommend relevant visualizations
   - 🔍 **Dashboard Discovery**: Explore available analytics content
   - 📈 **Context-Aware Suggestions**: Show dashboards based on user questions

4. **Integration Patterns**:
   ```python
   # Basic pattern
   "parameters": {
       "index": ".kibana",
       "input": "${parameters.question}",
       "size": 3
   }
   
   # With custom kibana index
   "parameters": {
       "index": ".kibana_custom",
       "input": "revenue dashboard",
       "size": 5
   }
   ```

### Best Practices:

- ✅ **Descriptive Titles**: Use clear, searchable visualization titles
- ✅ **Rich Descriptions**: Add detailed descriptions with keywords
- ✅ **Tagging**: Include relevant tags in visualization metadata
- ✅ **Size Tuning**: Adjust `size` parameter based on use case
- ✅ **Index Configuration**: Ensure .kibana index has proper mappings

### Dashboard Organization Tips:

- 🏷️ **Naming Convention**: Use consistent, descriptive names
- 📝 **Metadata**: Fill in all description fields
- 🎯 **Purpose-Driven**: Create dashboards for specific questions
- 👥 **Audience-Specific**: Tailor dashboards to user roles
- 🔄 **Regular Updates**: Keep dashboards current and relevant

### Combining with Other Tools:

```python
# Example: Multi-tool agent
tools = [
    {"type": "VisualizationTool", ...},  # Find relevant dashboards
    {"type": "SearchIndexTool", ...},    # Get underlying data
    {"type": "MLModelTool", ...}         # Generate insights
]
```

---

## 🧹 Cleanup (Optional)

Uncomment and run this cell to clean up resources created in this notebook.

In [ ]:
# # Delete the flow agent
# cleanup_resources(
#     client=client,
#     agent_ids=[agent_id]
# )

# # Delete demo index
# client.indices.delete(index=kibana_index, ignore=[404])

# print("✅ Cleanup complete!")

## 🚀 Next Steps

Now that you understand VisualizationTool, explore:
- **SearchIndexTool**: Query the underlying data for visualizations
- **RAGTool**: Generate insights based on visualization data
- **MLModelTool**: Add AI-powered analysis to dashboard discovery
- **AgentTool**: Combine visualization discovery with data analysis

---

📚 **Resources**:
- [OpenSearch Dashboards Documentation](https://opensearch.org/docs/latest/dashboards/)
- [Saved Objects API](https://opensearch.org/docs/latest/dashboards/dev-tools/saved-objects/)
- [Visualization Types](https://opensearch.org/docs/latest/dashboards/visualize/viz-index/)
- [ML Commons Agent Tools](https://opensearch.org/docs/latest/ml-commons-plugin/agents-tools/)